In [1]:
import cv2
import json
import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [2]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 1280
PATH_GT = 'test_label.json'
PATH_TEST = 'test_set/'
ROI = 160

In [3]:
def segment(img, cut_height, cut_width, show=False):
    roi_height = IMAGE_HEIGHT - ROI
    
    near = np.array(img)
    for i in range(roi_height):
        for j in range(roi_height - i):
            near[ROI + i][j] = 0
    for i in range(roi_height):
        for j in range(IMAGE_WIDTH - roi_height + i, IMAGE_WIDTH):
            near[ROI + i][j] = 0
            
    far = np.bitwise_xor(img, near)
    for i in range(cut_height):
        for j in range(cut_height - i):
            near[IMAGE_HEIGHT - i - 1][int(IMAGE_WIDTH / 2 - cut_width / 2 - j - 1)] = 0
    for i in range(cut_height):
        for j in range(cut_width):
            near[IMAGE_HEIGHT - i - 1][int(IMAGE_WIDTH / 2 - cut_width / 2 + j)] = 0
    for i in range(cut_height):
        for j in range(cut_height - i):
            near[IMAGE_HEIGHT - i - 1][int(IMAGE_WIDTH / 2 + cut_width / 2 + j)] = 0

    if show:
        plt.imshow(img, cmap='gray')
        plt.show()

        plt.imshow(near, cmap='gray')
        plt.show()

        plt.imshow(far, cmap='gray')
        plt.show()
            
    return near, far

In [4]:
def cluster(img, lines, coords, thresh, mode='intercept', show=False):
    lines_clustered = []
    img_2 = np.array(img)
    if lines.shape[0] > 1:
        if mode == 'intercept':
            X = lines[:, 1].reshape(-1, 1)
        else:
            X = lines[:, 0].reshape(-1, 1)
        clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=thresh).fit(X)
        for i in range(clustering.n_clusters_):
            ind = np.argwhere(clustering.labels_ == i)
            seg = coords[ind].reshape(-1, 4)
            fit = np.polyfit(seg[:, 0:3:2].reshape(-1), seg[:, 1:4:2].reshape(-1), 1)
            x1 = int((160 - fit[1]) / fit[0])
            x2 = int((710 - fit[1]) / fit[0])
            cv2.line(img_2, (x1, 160), (x2, 710), (0, 255, 0), 2)
            lines_clustered.append(fit)
    elif lines.shape[0] == 1:
        x1 = int((160 - lines[0][1]) / lines[0][0])
        x2 = int((710 - lines[0][1]) / lines[0][0])
        cv2.line(img_2, (x1, 160), (x2, 710), (0, 255, 0), 2)
        lines_clustered.append(lines[0])

    if show:
        img_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2RGB)
        plt.imshow(img_2)
        plt.show()
        
    return np.array(lines_clustered).reshape((-1, 2))

In [5]:
def hough(img, edges, minLineLength, maxLineGap, grad_min, grad_max, thresh, show=False):
    img_2 = np.array(img)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, minLineLength=minLineLength, maxLineGap=maxLineGap)
    lines_valid = []
    coord_valid = []
    if lines is None:
        return np.empty((0, 2))
    for line in lines:
        x1, y1, x2, y2 = line[0]
        centerx = (x2 + x1) / 2
        fit = np.polyfit([x1, x2], [y1, y2], 1)
        grad = fit[0]
        if centerx < 640 and grad < -grad_min and grad > -grad_max:
            coord_valid.append(line[0])
            lines_valid.append(fit)
            cv2.line(img_2, (x1, y1), (x2, y2), (0, 255, 0), 2)
        elif centerx >= 640 and grad > grad_min and grad < grad_max:
            coord_valid.append(line[0])
            lines_valid.append(fit)
            cv2.line(img_2, (x1, y1), (x2, y2), (0, 255, 0), 2)
    lines_valid = np.array(lines_valid)
    coord_valid = np.array(coord_valid)
    
    if show:
        img_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2RGB)
        plt.imshow(img_2)
        plt.show()
        
    return cluster(img, lines_valid, coord_valid, thresh, show=show)

In [6]:
def get_lanes(img_path, show=False):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150)
    for i in range(ROI):
        for j in range(IMAGE_WIDTH):
            edges[i][j] = 0
    near, far = segment(edges, 250, 300, show=show)
    lines_near = hough(img, near, 100, 10, 0.7, 1.6, 800, show=show)
    lines_far = hough(img, far, 120, 5, 0.2, 0.7, 100, show=show)
    lines = np.concatenate((lines_near, lines_far))
    limit = 160
    for i in range(lines.shape[0]):
        i_m = lines[i][0]
        i_c = lines[i][1]
        for j in range(i + 1, lines.shape[0]):
            j_m = lines[j][0]
            j_c = lines[j][1]
            intercept = (j_m * i_c - i_m * j_c) / (j_m - i_m)
            if intercept > limit:
                limit = intercept
    limit = min(math.ceil(limit / 10) * 10, 710)
    coords = np.full((lines.shape[0], 56), -2)
    for i in range(lines.shape[0]):
        for j in range(limit, 711, 10):
            index = int((j - 160) / 10)
            pt = (j - lines[i][1]) / lines[i][0]
            if pt >= 0 and pt < IMAGE_WIDTH:
                coords[i][index] = (j - lines[i][1]) / lines[i][0]
    for i in coords:
        index = int((limit - 160) / 10)
        x1 = i[index]
        end = int((710 - 160) / 10)
        while (end >= 0 and i[end] == -2):
            end -= 1
        if end >= 0:
            x2 = i[end]
            y2 = end * 10 + 160
            cv2.line(img, (x1, limit), (x2, y2), (0, 255, 0), 2)
        
    if show:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.show()
        
    return coords

In [7]:
lr = LinearRegression()
pixel_thresh = 20
pt_thresh = 0.85

def get_angle(xs, y_samples):
    xs, ys = xs[xs >= 0], y_samples[xs >= 0]
    if len(xs) > 1:
        lr.fit(ys[:, None], xs)
        k = lr.coef_[0]
        theta = np.arctan(k)
    else:
        theta = 0
    return theta

def line_accuracy(pred, gt, thresh):
    pred = np.array([p if p >= 0 else -100 for p in pred])
    gt = np.array([g if g >= 0 else -100 for g in gt])
    return np.sum(np.where(np.abs(pred - gt) < thresh, 1., 0.)) / len(gt)

def bench(pred, gt, y_samples):
    if any(len(p) != len(y_samples) for p in pred):
        raise Exception('Format of lanes error.')
    if len(gt) + 2 < len(pred):
        return 0., 0., 1.
    angles = [get_angle(np.array(x_gts), np.array(y_samples)) for x_gts in gt]
    threshs = [pixel_thresh / np.cos(angle) for angle in angles]
    line_accs = []
    fp, fn = 0., 0.
    matched = 0.
    for x_gts, thresh in zip(gt, threshs):
        accs = [line_accuracy(np.array(x_preds), np.array(x_gts), thresh) for x_preds in pred]
        max_acc = np.max(accs) if len(accs) > 0 else 0.
        if max_acc < pt_thresh:
            fn += 1
        else:
            matched += 1
        line_accs.append(max_acc)
    fp = len(pred) - matched
    if len(gt) > 4 and fn > 0:
        fn -= 1
    s = sum(line_accs)
    if len(gt) > 4:
        s -= min(line_accs)
    return s / max(min(4.0, len(gt)), 1.), fp / len(pred) if len(pred) > 0 else 0., fn / max(min(len(gt), 4.) , 1.)

In [8]:
def test_lanes():
    gts = [json.loads(line) for line in open(PATH_GT)]
    accuracy, fp, fn = 0., 0., 0.
    num = len(gts)
    for gt in gts:
        gt_lanes = gt['lanes']
        y_samples = gt['h_samples']
        img_path = f'{PATH_TEST}{gt["raw_file"]}'
        pred_lanes = get_lanes(img_path).tolist()
        a, p, n = bench(pred_lanes, gt_lanes, y_samples)
        accuracy += a
        fp += p
        fn += n
    return accuracy / num, fp / num, fn / num

In [9]:
accuracy, fp, fn = test_lanes()
print(f'Accuracy: {accuracy}')
print(f'False Positive: {fp}')
print(f'False Negative: {fn}')

<ipython-input-6-9f73b4aa8efb>:10: RankWarning: Polyfit may be poorly conditioned
  lines_near = hough(img, near, 100, 10, 0.7, 1.6, 800, show=show)
<ipython-input-6-9f73b4aa8efb>:10: RankWarning: Polyfit may be poorly conditioned
  lines_near = hough(img, near, 100, 10, 0.7, 1.6, 800, show=show)
<ipython-input-6-9f73b4aa8efb>:10: RankWarning: Polyfit may be poorly conditioned
  lines_near = hough(img, near, 100, 10, 0.7, 1.6, 800, show=show)
<ipython-input-6-9f73b4aa8efb>:11: RankWarning: Polyfit may be poorly conditioned
  lines_far = hough(img, far, 120, 5, 0.2, 0.7, 100, show=show)
<ipython-input-6-9f73b4aa8efb>:11: RankWarning: Polyfit may be poorly conditioned
  lines_far = hough(img, far, 120, 5, 0.2, 0.7, 100, show=show)
<ipython-input-6-9f73b4aa8efb>:10: RankWarning: Polyfit may be poorly conditioned
  lines_near = hough(img, near, 100, 10, 0.7, 1.6, 800, show=show)
<ipython-input-6-9f73b4aa8efb>:10: RankWarning: Polyfit may be poorly conditioned
  lines_near = hough(img, near

Accuracy: 0.6545752242305983
False Positive: 0.46683098832631403
False Negative: 0.5593098490294731
